In [1]:
import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
original_dataset_dir = '/home/pzhang/Downloads/ml_datasets/cat-and-dog'

base_dir = '/home/pzhang/Downloads/ml_datasets/cat-and-dog-small'

if not os.path.exists(base_dir):
    os.mkdir(base_dir)

In [8]:
!ls /home/pzhang/Downloads/ml_datasets/cat-and-dog/test_set

cats  dogs


In [10]:
!ls /home/pzhang/Downloads/ml_datasets/cat-and-dog/test_set/cats | wc -l
!ls /home/pzhang/Downloads/ml_datasets/cat-and-dog/test_set/dogs | wc -l

1012
1013


测试集中约有2000个样本

In [12]:
!ls /home/pzhang/Downloads/ml_datasets/cat-and-dog/training_set/cats | wc -l
!ls /home/pzhang/Downloads/ml_datasets/cat-and-dog/training_set/dogs | wc -l

4001
4006


 训练集大概有 8000 个样本

## 划分验证集

In [26]:
train_path = os.path.join(original_dataset_dir, 'training_set')
test_path = os.path.join(original_dataset_dir, 'test_set')

train_path_cats = os.path.join(train_path, 'cats')
train_path_dogs = os.path.join(train_path, 'dogs')

path_cats = [
    os.path.join(train_path_cats, path) for path in os.listdir(train_path_cats)
    if '.jpg' in path
]
path_dogs = [
    os.path.join(train_path_dogs, path) for path in os.listdir(train_path_dogs)
    if '.jpg' in path
]

len(path_cats), len(path_dogs)

(4000, 4005)

将训练集8000个样本中的6000做训练集，余下的2000个样本当做验证集。需要注意的是，训练和验证集中`cat`和`dog`的比例均为50%。

In [18]:
train_set = np.zeros((6000, 150, 150, 3), dtype=np.float32)


In [19]:
from keras.preprocessing.image import ImageDataGenerator


## 数据预处理

大概分为以下几个步骤

1. 读取图像文件
 